# Taguchi Array

In [2]:
import numpy as np

In [3]:
# Input array
z_sh = [ 7, 10,  5 ]
z_p2 = [30, 32, 28 ]
z_r2 = [80, 74, 82 ]
x_s = [ 0.5,  0.4,  0.3 ]
x_p1 = [ 1.0, 0.9, 0.8 ]
x_p2 = [-0.3, 0.4, 0.5 ]
x_r2 = [ 0.477546, 0.377546, 0.277546 ]
gear_thickness = [6,8,12]

param_vals = [z_sh, z_p2, z_r2, x_s, x_p1, x_p2, x_r2, gear_thickness]

paper_vals = [6, 32, 81, .476, .762, .536, 1.21, 10] # NOTE - 6 = 12 (z_s) / 2.

for i in range(len(param_vals)):
    # append paper_val to the front of the list
    param_vals[i].insert(0, paper_vals[i])

In [4]:
Taguchi = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
[1, 1, 2, 2, 2, 2, 2, 2, 2, 2],
[1, 1, 3, 3, 3, 3, 3, 3, 3, 3],
[1, 1, 4, 4, 4, 4, 4, 4, 4, 4],
[1, 2, 1, 1, 2, 2, 3, 3, 4, 4],
[1, 2, 2, 2, 1, 1, 4, 4, 3, 3],
[1, 2, 3, 3, 4, 4, 1, 1, 2, 2],
[1, 2, 4, 4, 3, 3, 2, 2, 1, 1],
[1, 3, 1, 2, 3, 4, 1, 2, 3, 4],
[1, 3, 2, 1, 4, 3, 2, 1, 4, 3],
[1, 3, 3, 4, 1, 2, 3, 4, 1, 2],
[1, 3, 4, 3, 2, 1, 4, 3, 2, 1],
[1, 4, 1, 2, 4, 3, 3, 4, 2, 1],
[1, 4, 2, 1, 3, 4, 4, 3, 1, 2],
[1, 4, 3, 4, 2, 1, 1, 2, 4, 3],
[1, 4, 4, 3, 1, 2, 2, 1, 3, 4],
[2, 1, 1, 4, 1, 4, 2, 3, 2, 3],
[2, 1, 2, 3, 2, 3, 1, 4, 1, 4],
[2, 1, 3, 2, 3, 2, 4, 1, 4, 1],
[2, 1, 4, 1, 4, 1, 3, 2, 3, 2],
[2, 2, 1, 4, 2, 3, 4, 1, 3, 2],
[2, 2, 2, 3, 1, 4, 3, 2, 4, 1],
[2, 2, 3, 2, 4, 1, 2, 3, 1, 4],
[2, 2, 4, 1, 3, 2, 1, 4, 2, 3],
[2, 3, 1, 3, 3, 1, 2, 4, 4, 2],
[2, 3, 2, 4, 4, 2, 1, 3, 3, 1],
[2, 3, 3, 1, 1, 3, 4, 2, 2, 4],
[2, 3, 4, 2, 2, 4, 3, 1, 1, 3],
[2, 4, 1, 3, 4, 2, 4, 2, 1, 3],
[2, 4, 2, 4, 3, 1, 3, 1, 2, 4],
[2, 4, 3, 1, 2, 4, 2, 4, 3, 1],
[2, 4, 4, 2, 1, 3, 1, 3, 4, 2],
])-1

In [5]:
first_set = []
Tag_vals = Taguchi[0][1:-1]
print(Tag_vals)
for i in range(len(param_vals)):
    first_set.append(param_vals[i][Tag_vals[i]])

[0 0 0 0 0 0 0 0]


In [6]:
first_set

[6, 32, 81, 0.476, 0.762, 0.536, 1.21, 10]

In [8]:
targ_set = 2 # 1-32
new_set = []
Tag_vals = Taguchi[targ_set-1][1:-1]
# print(Tag_vals)
for i in range(len(param_vals)):
    new_set.append(param_vals[i][Tag_vals[i]])
new_set

[6, 30, 80, 0.5, 1.0, -0.3, 0.477546, 6]

In [9]:
for targ_set in range(len(Taguchi)):
    new_set = []
    Tag_vals = Taguchi[targ_set][1:-1]
    # print(Tag_vals)
    for i in range(len(param_vals)):
        new_set.append(param_vals[i][Tag_vals[i]])
    
    # Print the new set of parameters
    print(f"Set {targ_set + 1}: {new_set}")
    
    # Here you can add code to perform calculations or simulations with new_set if needed

Set 1: [6, 32, 81, 0.476, 0.762, 0.536, 1.21, 10]
Set 2: [6, 30, 80, 0.5, 1.0, -0.3, 0.477546, 6]
Set 3: [6, 32, 74, 0.4, 0.9, 0.4, 0.377546, 8]
Set 4: [6, 28, 82, 0.3, 0.8, 0.5, 0.277546, 12]
Set 5: [7, 32, 81, 0.5, 1.0, 0.4, 0.377546, 12]
Set 6: [7, 30, 80, 0.476, 0.762, 0.5, 0.277546, 8]
Set 7: [7, 32, 74, 0.3, 0.8, 0.536, 1.21, 6]
Set 8: [7, 28, 82, 0.4, 0.9, -0.3, 0.477546, 10]
Set 9: [10, 32, 80, 0.4, 0.8, 0.536, 0.477546, 8]
Set 10: [10, 30, 81, 0.3, 0.9, -0.3, 1.21, 12]
Set 11: [10, 32, 82, 0.476, 1.0, 0.4, 0.277546, 10]
Set 12: [10, 28, 74, 0.5, 0.762, 0.5, 0.377546, 6]
Set 13: [5, 32, 80, 0.3, 0.9, 0.4, 0.277546, 6]
Set 14: [5, 30, 81, 0.4, 0.8, 0.5, 0.377546, 10]
Set 15: [5, 32, 82, 0.5, 0.762, 0.536, 0.477546, 12]
Set 16: [5, 28, 74, 0.476, 1.0, -0.3, 1.21, 8]
Set 17: [6, 32, 82, 0.476, 0.8, -0.3, 0.377546, 6]
Set 18: [6, 30, 74, 0.5, 0.9, 0.536, 0.277546, 10]
Set 19: [6, 32, 80, 0.4, 1.0, 0.5, 1.21, 12]
Set 20: [6, 28, 81, 0.3, 0.762, 0.4, 0.477546, 8]
Set 21: [7, 32, 82, 